In [1]:
from google.colab import files
uploaded = files.upload()

Saving fake_news.csv to fake_news.csv
Saving real_news.csv to real_news.csv


In [6]:
import pandas as pd
import re
import numpy as np
import sklearn


## **df1 dataframe for reading fake news and df2 dataframe for reading true news**

In [7]:
import io
df1=pd.read_csv(io.BytesIO(uploaded['fake_news.csv']))
df2=pd.read_csv(io.BytesIO(uploaded['real_news.csv']))

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [10]:
df1.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


In [11]:
df2.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [13]:
#value classification
df1["class"]=0
df2["class"]=1

In [14]:
df1.shape, df2.shape

((23481, 5), (21417, 5))

In [16]:
#using last 10 rows
df1_manual_testing = df1.tail(10)
for i in range(23480,23471,-1):
  df1.drop([i],axis=0,inplace=True)
df2_manual_testing = df2.tail(10)
for i in range(21416,21406,-1):
  df2.drop([i],axis=0,inplace=True)

In [18]:
#saving the testing files as a csv file
df_testing = pd.concat([df1_manual_testing,df2_manual_testing],axis=0)
df_testing.to_csv("phy_testing.csv")

In [19]:
df_combined= pd.concat([df1,df2],axis=0)
df_combined.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


* Title, Subject and Date columns are not necessary and can be removed*

In [20]:
df_detect = df_combined.drop(["title","subject","date"],axis=1)
#axis=1 for removal of columns

In [22]:
df_detect = df_detect.sample(frac=1)
#for shuffling of the dataset values

In [23]:
df_detect.head(10)

,text,class
1113,WASHINGTON (Reuters) - The U.S. House of Repre...,1
1612,WASHINGTON (Reuters) - Republican leaders soug...,1
19772,Hillary s such a champion of gay rights that h...,0
11416,BEIJING (Reuters) - China called on Monday for...,1
20177,BERLIN (Reuters) - Nihan Sen s grandmother ca...,1
20439,The left wants to change hurtful words to wh...,0
2149,"Donald Trump just got owned.On Monday, the Con...",0
18525,"COPENHAGEN (Reuters) - Denmark, part of the NA...",1
10205,A University of Georgia professor has adopted ...,0
16181,"BAGHDAD (Reuters) - Demonstrators, some carryi...",1


In [24]:
df_detect.isnull().sum()

text     0
class    0
dtype: int64

# removal of special characters and other unnecessary characters

In [29]:
def word_drop(text):
  text= text.lower()
  text= re.sub('\[.*?\]','',text)
  text= re.sub("\\W"," ",text)
  text= re.sub('https?://\S+|www\.\S+','',text)
  text= re.sub('<.*?>+','',text)
  text= re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text= re.sub('\n','',text)
  text= re.sub('\w*\d\w*','',text)
  return text

In [30]:
df_detect["text"]= df_detect["text"].apply(word_drop)

In [32]:
x = df_detect["text"]
y = df_detect["class"]

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

In [37]:
#we use Tfidf Vectorizer
vectorization = TfidfVectorizer()
x1_train = vectorization.fit_transform(x_train)
x1_test = vectorization.transform(x_test)

## ***LOGISTIC REGRESSION***

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
lr = LogisticRegression()
lr.fit(x1_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
lr.score(x1_test,y_test)

0.9858288770053476

*model is 98.5% accurate*

In [42]:
#Classification report
pred_lr = lr.predict(x1_test)

In [43]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5810
           1       0.98      0.99      0.99      5410

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



# *DECISION TREE CLASSIFICATION*

In [44]:
from sklearn.tree import DecisionTreeClassifier

In [45]:
dt = DecisionTreeClassifier()
dt.fit(x1_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [46]:
dt.score(x1_test,y_test)

0.9957219251336898

99.5% accurate the model

In [47]:
dt_pred = dt.predict(x1_test)

In [48]:
print(classification_report(y_test,dt_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5810
           1       1.00      1.00      1.00      5410

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



## *GRADIENT BOOST CLASSIFIFER*

In [49]:
from sklearn.ensemble import GradientBoostingClassifier

In [51]:
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(x1_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [52]:
gbc.score(x1_test,y_test)

0.995632798573975

In [53]:
gbc_pred = gbc.predict(x1_test)

In [54]:
print(classification_report(y_test,gbc_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5810
           1       0.99      1.00      1.00      5410

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



## *RANDOM FOREST CLASSIFIER*

In [56]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(x1_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [58]:
rfc.score(x1_test,y_test)

0.9877896613190731

98.7% accurate model

In [59]:
rfc_pred = rfc.predict(x1_test)

In [60]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5810
           1       0.99      0.99      0.99      5410

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



# *Manual Testing*

In [66]:
def op_label(n):
  if n == 0:
    return "Fake news Detected."
  elif n==1:
    return "News Detected not fake."

def manual_testing(nws):
  test_nws = {"text":[nws]}
  new_test = pd.DataFrame(test_nws)
  new_test["text"] = new_test["text"].apply(word_drop)
  new_xtest = new_test["text"]
  new_x1test = vectorization.transform(new_xtest)
  lr_pred = lr.predict(new_x1test)
  dt_pred = dt.predict(new_x1test)
  gbc_pred = gbc.predict(new_x1test)
  rfc_pred = rfc.predict(new_x1test)

  return print("\n Decision Tree Prediction: {} \n Logistic Regression Prediction: {} \n Gradient Boosting Classifier Prediction: {} \n Random Forest Classifier Prediction: {}".format(op_label(dt_pred[0]),op_label(lr_pred[0]),op_label(gbc_pred[0]),op_label(rfc_pred[0])
                                                                                                                                                                                        ))

In [68]:
news = str(input())
manual_testing(news)

 JAKARTA (Reuters) - Indonesia will buy 11 Sukhoi fighter jets worth $1.14 billion from Russia in exchange for cash and Indonesian commodities, two cabinet ministers said on Tuesday. The Southeast Asian country has pledged to ship up to $570 million worth of commodities in addition to cash to pay for the Suhkoi SU-35 fighter jets, which are expected to be delivered in stages starting in two years. Indonesian Trade Minister Enggartiasto Lukita said in a joint statement with Defence Minister Ryamizard Ryacudu that details of the type and volume of commodities were  still being negotiated . Previously he had said the exports could include palm oil, tea, and coffee. The deal is expected to be finalised soon between Indonesian state trading company PT Perusahaan Perdangangan Indonesia and Russian state conglomerate Rostec. Russia is currently facing a new round of U.S.-imposed trade sanctions. Meanwhile, Southeast Asia s largest economy is trying to promote its palm oil products amid threat